In [1]:
import os
import pandas as pd

Dataset 1

In [2]:
# Set the path to the target directory
base_path = "C:/Users/Ally/Desktop/sheng/data/1"

# Get a list of all subdirectories in "1"
subdirs = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

# Print each subdirectory name
for subdir in subdirs:
    print(subdir)

AEROBIC
ANAEROBIC
STRESS
wearable-device-dataset-from-induced-stress-and-structured-exercise-sessions-1.0.0


In [3]:
i = 0

In [4]:
dir_name = subdirs[i]
dir_path = os.path.join(base_path, dir_name)

In [5]:
subject_dirs = [name for name in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, name))]

In [6]:
subject_dirs

['f01',
 'f02',
 'f03',
 'f04',
 'f05',
 'f06',
 'f07',
 'f08',
 'f09',
 'f10',
 'f11',
 'f12',
 'f13',
 'S01',
 'S02',
 'S03',
 'S04',
 'S05',
 'S06',
 'S07',
 'S08',
 'S09',
 'S10',
 'S11_a',
 'S11_b',
 'S13',
 'S14',
 'S15',
 'S16',
 'S17',
 'S18']

In [7]:
j = 0


In [8]:
subject_dirs[j]
os.listdir(os.path.join(dir_path, subject_dirs[j]))

['ACC.csv', 'BVP.csv', 'EDA.csv', 'HR.csv', 'IBI.csv', 'tags.csv', 'TEMP.csv']

In [9]:
import pandas as pd

In [10]:
BVP_path = os.path.join(dir_path, subject_dirs[j], "BVP.csv")
EDA_path = os.path.join(dir_path, subject_dirs[j], "EDA.csv")
TEMP_path = os.path.join(dir_path, subject_dirs[j], "TEMP.csv")

In [11]:
# Read the CSV file
df_BVP = pd.read_csv(BVP_path, header=None)
df_EDA = pd.read_csv(EDA_path, header=None)
df_TEMP = pd.read_csv(TEMP_path, header=None)

In [12]:
# Display the first few rows
print(df_BVP.head())

                     0
0  2013-07-04 12:06:34
1                 64.0
2                 -0.0
3                 -0.0
4                 -0.0


In [13]:
df_BVP.iloc[[0]]

,0
0,2013-07-04 12:06:34


In [14]:
df_BVP.shape

(172702, 1)

In [15]:
df_BVP.iloc[0,0]

'2013-07-04 12:06:34'

In [16]:
df_EDA.iloc[0,0]

'2013-07-04 12:06:34'

In [17]:
df_BVP.iloc[0,0] == df_EDA.iloc[0,0]

True

In [18]:
df_BVP.iloc[0,0] == df_TEMP.iloc[0,0]

True

In [19]:
df_BVP = df_BVP.iloc[2:].reset_index(drop=True)
df_EDA = df_EDA.iloc[2:].reset_index(drop=True)
df_TEMP = df_TEMP.iloc[2:].reset_index(drop=True)

In [20]:
df_BVP.shape

(172700, 1)

In [21]:
df_EDA.shape

(10794, 1)

In [22]:
df_TEMP.shape

(10792, 1)

In [23]:
# downsample BVP to 32 hz
df_BVP = df_BVP.iloc[::2].reset_index(drop=True)

In [24]:
df_EDA = df_EDA.loc[df_EDA.index.repeat(8)].reset_index(drop=True)
df_TEMP = df_TEMP.loc[df_TEMP.index.repeat(8)].reset_index(drop=True)

In [25]:
# combine all three df into one data frame
# Step 1: Find the minimum number of rows
min_rows = min(len(df_BVP), len(df_EDA), len(df_TEMP))

# Step 2: Cut each dataframe to the minimum number of rows
df_BVP_cut = df_BVP.iloc[:min_rows].reset_index(drop=True)
df_EDA_cut = df_EDA.iloc[:min_rows].reset_index(drop=True)
df_TEMP_cut = df_TEMP.iloc[:min_rows].reset_index(drop=True)

In [26]:
df= pd.concat([df_BVP_cut, df_EDA_cut, df_TEMP_cut], axis=1)

In [27]:
df.columns = ['BVP', 'EDA', 'TEMP']

In [28]:
df

,BVP,EDA,TEMP
0,-0.0,0.0,31.35
1,-0.0,0.0,31.35
2,-0.0,0.0,31.35
3,-0.0,0.0,31.35
4,-0.0,0.0,31.35
...,...,...,...
86331,-23.15,0.0,28.17
86332,-6.12,0.0,28.17
86333,2.65,0.0,28.17
86334,-1.08,0.0,28.17


In [29]:
subject_dirs[j]

'f01'

In [30]:
df = df.reset_index().rename(columns={'index': 'time'})

In [31]:
df['subject'] = subject_dirs[j]

In [32]:
df

,time,BVP,EDA,TEMP,subject
0,0,-0.0,0.0,31.35,f01
1,1,-0.0,0.0,31.35,f01
2,2,-0.0,0.0,31.35,f01
3,3,-0.0,0.0,31.35,f01
4,4,-0.0,0.0,31.35,f01
...,...,...,...,...,...
86331,86331,-23.15,0.0,28.17,f01
86332,86332,-6.12,0.0,28.17,f01
86333,86333,2.65,0.0,28.17,f01
86334,86334,-1.08,0.0,28.17,f01


In [34]:
# Parameters
fs = 32              # Sampling frequency (Hz)
chunk_duration = 4   # seconds
overlap_duration = 1 # seconds

chunk_size = chunk_duration * fs
overlap_size = overlap_duration * fs
step_size = chunk_size - overlap_size  # How many rows to move forward for each new chunk

# Store all chunks
chunks = []

# Go through the DataFrame
for start in range(0, len(df) - chunk_size + 1, step_size):
    chunk = df.iloc[start : start + chunk_size].reset_index(drop=True)
    chunks.append(chunk)

# Now `chunks` is a list of DataFrames, each 128 rows, overlapping by 32 rows


In [38]:
len(chunks)

899